# <font color=teal>imports</font>
Most processing is performed in python code, and there's a python module to do everything here without manual

In [1]:
import logging

from src import configs
import os
import sys
sys.path.append(os.path.abspath("../src"))



Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
from src.nflverse_transform_job import load_files
from src.pbp_fact import transform_pbp
from src.nflverse_reader_job import URLReader
from src.pbp_participation import transform_pbp_participation
from src.player_stats import transform_player_stats, merge_injuries
from src.player_injuries import prep_player_injuries
from src.player_stats import transform_players
from src.db_utils import load_dims_to_db


# <font color=teal>housekeeping</font>

In [3]:
LOAD_TO_DB = True
database_schema = 'controls'

# Get the logger
logger = configs.configure_logging("pbp_logger")
logger.setLevel(logging.INFO)

# <font color=teal>read data from nflverse<font/>
Step 1.  Read data and store immediately as raw without transformation or change

In [4]:
%%time
reader = URLReader(start_year=2017, last_year=2022, file_type='parquet')
urls = reader.download()


2023-07-13 13:31:35,355 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_passing.csv.gz
2023-07-13 13:31:35,435 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/pbp_participation/pbp_participation_2017.parquet
2023-07-13 13:31:35,509 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_receiving.csv.gz
2023-07-13 13:31:35,629 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/nextgen_stats/ngs_2016_rushing.csv.gz
2023-07-13 13:31:35,718 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/injuries/injuries_2016.parquet
2023-07-13 13:31:36,009 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/pbp_participation/pbp_participation_2016.parquet
2023-07-13 13:31:36,046 - INFO - Success: https://github.com/nflverse/nflverse-data/releases/download/players/players.parquet
2023-07-13 13:31:

CPU times: user 12.2 s, sys: 542 ms, total: 12.7 s
Wall time: 11.6 s


---

# <font color=teal>load and transform play by play datasets</font>

### <font color="#9370DB">load</font>

In [5]:

%%time
pbp = load_files(data_subdir='pbp')


2023-07-13 13:31:45,125 - INFO - Reading all files from pbp
2023-07-13 13:31:45,126 - INFO -   + Reading pbp_2019.parquet
2023-07-13 13:31:45,356 - INFO -   + Reading pbp_2018.parquet
2023-07-13 13:31:45,468 - INFO -   + Reading pbp_2022.parquet
2023-07-13 13:31:45,581 - INFO -   + Reading pbp_2021.parquet
2023-07-13 13:31:45,689 - INFO -   + Reading pbp_2017.parquet
2023-07-13 13:31:45,794 - INFO -   + Reading pbp_2016.parquet
2023-07-13 13:31:45,898 - INFO -   + Reading pbp_2020.parquet


CPU times: user 2.4 s, sys: 776 ms, total: 3.18 s
Wall time: 2.52 s


### <font color="#9370DB">transform</font>

In [6]:

%%time
datasets = transform_pbp(pbp)

2023-07-13 13:31:49,955 - INFO - Impute columns to 0
2023-07-13 13:31:50,163 - INFO - impute non binary pbp columns ...
2023-07-13 13:31:50,897 - INFO - Impute columns to 0
2023-07-13 13:31:51,493 - INFO - Impute columns to 0:00
2023-07-13 13:31:52,832 - INFO - Impute columns to NA
2023-07-13 13:31:58,505 - INFO - moving play_id to play_counter, and creating a joinable play_id key
2023-07-13 13:31:59,861 - INFO - Conform key actions like pass, rush, kickoff, etc. and add a single category field called actions... 
2023-07-13 13:32:06,182 - INFO - Validate actions dimension ...
2023-07-13 13:32:06,389 - INFO - Creating new drive dimension...
2023-07-13 13:32:06,460 - INFO - Validate drive_df dimension ...
2023-07-13 13:32:06,684 - INFO - Creating new situations dimension...
2023-07-13 13:32:06,750 - INFO - Validate situation_df dimension ...
2023-07-13 13:32:06,958 - INFO - Creating new metrics dimension...
2023-07-13 13:32:07,005 - INFO - Validate play_metrics_df dimension ...
2023-07-1

CPU times: user 17.5 s, sys: 5.65 s, total: 23.2 s
Wall time: 25.1 s


---

# <font color=teal>load and transform play by play participation datasets</font>

### <font color="#9370DB">load</font>

In [7]:
%%time
pbp_participation_df = load_files('pbp-participation')


2023-07-13 13:32:12,784 - INFO - Reading all files from pbp-participation
2023-07-13 13:32:12,786 - INFO -   + Reading pbp-participation_2019.parquet
2023-07-13 13:32:12,823 - INFO -   + Reading pbp-participation_2018.parquet
2023-07-13 13:32:12,852 - INFO -   + Reading pbp-participation_2017.parquet
2023-07-13 13:32:12,876 - INFO -   + Reading pbp-participation_2021.parquet
2023-07-13 13:32:12,901 - INFO -   + Reading pbp-participation_2020.parquet
2023-07-13 13:32:12,924 - INFO -   + Reading pbp-participation_2016.parquet
2023-07-13 13:32:12,946 - INFO -   + Reading pbp-participation_2022.parquet


CPU times: user 260 ms, sys: 123 ms, total: 382 ms
Wall time: 301 ms


### <font color="#9370DB">transform</font>

In [8]:
%%time
player_df, player_events_df = transform_pbp_participation(
    participation_df=pbp_participation_df,
    player_events=datasets['player_events'])

datasets.update({
    'player_participation': player_df,
    'player_events': player_events_df,
})

2023-07-13 13:32:13,088 - INFO - pbp_participation:  move play_id to a play_count column and create a unique play_id that can be used in joins...
2023-07-13 13:32:13,312 - INFO - Calculating defense and offense team names by player and play...
2023-07-13 13:32:15,882 - INFO - Exploding offensive players to their own dataset...
2023-07-13 13:32:17,415 - INFO - Exploding defense_players to their own dataset...
2023-07-13 13:32:26,121 - WARNING - combining offense and defense players - counts are incorrect 6903461 != 6902991


---

# <font color=teal>transform player injuries</font>

### <font color="#9370DB">load</font>

In [9]:
%%time
injuries_df = load_files('injuries')

2023-07-13 13:32:30,350 - INFO - Reading all files from injuries
2023-07-13 13:32:30,351 - INFO -   + Reading injuries_2017.parquet
2023-07-13 13:32:30,358 - INFO -   + Reading injuries_2021.parquet
2023-07-13 13:32:30,363 - INFO -   + Reading injuries_2020.parquet
2023-07-13 13:32:30,370 - INFO -   + Reading injuries_2016.parquet
2023-07-13 13:32:30,375 - INFO -   + Reading injuries_2022.parquet
2023-07-13 13:32:30,380 - INFO -   + Reading injuries_2019.parquet
2023-07-13 13:32:30,385 - INFO -   + Reading injuries_2018.parquet


CPU times: user 46.7 ms, sys: 15.3 ms, total: 61.9 ms
Wall time: 50.7 ms


### <font color="#9370DB">transform</font>

In [10]:
%%time
injuries_df = prep_player_injuries(injuries_df)

2023-07-13 13:32:30,403 - INFO - Prep injury data...
2023-07-13 13:32:30,404 - INFO - Conforming names (e.g. gsis_id -> player_id)
2023-07-13 13:32:30,415 - INFO - Merge sparse injury columns
2023-07-13 13:32:30,417 - INFO - Get best values for null report_statuses...
2023-07-13 13:32:30,488 - INFO - check that all positions are correct...


CPU times: user 83.9 ms, sys: 6.37 ms, total: 90.3 ms
Wall time: 93 ms


---

# <font color=teal>transform player stats</font>

In [11]:
%%time
stats_df = load_files('player-stats')
stats_df = transform_player_stats(stats_df)
stats_df = merge_injuries(player_stats=stats_df, player_injuries=injuries_df)

2023-07-13 13:32:30,499 - INFO - Reading all files from player-stats
2023-07-13 13:32:30,500 - INFO -   + Reading player-stats.parquet
2023-07-13 13:32:30,552 - INFO - fix specific player_stats: <function player_stats_fixes at 0x2d4755e50>..
2023-07-13 13:32:30,632 - INFO - replace empty position_groups with position info...
2023-07-13 13:32:30,646 - INFO - replace empty player_name with player_display_name info...
2023-07-13 13:32:30,658 - INFO - replace empty headshot_url with 'none'...
2023-07-13 13:32:30,669 - INFO - fillna(0) for all binary columns...
2023-07-13 13:32:30,669 - INFO - Impute columns to 0
2023-07-13 13:32:30,880 - WARNING - After merge player_stats count changed - went from 124081 to 124083


CPU times: user 337 ms, sys: 102 ms, total: 440 ms
Wall time: 385 ms


---

# <font color=teal>direct loads </font>

### <font color="#9370DB">adv stats</font>

In [12]:
%%time

advstats_def_df = load_files('advstats-season-def')
advstats_pass_df = load_files('advstats-season-pass')
advstats_rec_df = load_files('advstats-season-rec')
advstats_rush_df = load_files('advstats-season-rush')


2023-07-13 13:32:30,887 - INFO - Reading all files from advstats-season-def
2023-07-13 13:32:30,888 - INFO -   + Reading advstats-season-def.parquet
2023-07-13 13:32:30,893 - INFO - Reading all files from advstats-season-pass
2023-07-13 13:32:30,894 - INFO -   + Reading advstats-season-pass.parquet
2023-07-13 13:32:30,897 - INFO - Reading all files from advstats-season-rec
2023-07-13 13:32:30,897 - INFO -   + Reading advstats-season-rec.parquet
2023-07-13 13:32:30,901 - INFO - Reading all files from advstats-season-rush
2023-07-13 13:32:30,902 - INFO -   + Reading advstats-season-rush.parquet


CPU times: user 17.7 ms, sys: 6.8 ms, total: 24.5 ms
Wall time: 17.9 ms


### <font color="#9370DB">nextgen stats</font>

In [13]:
%%time
next_pass_df = load_files('nextgen-passing')


2023-07-13 13:32:30,910 - INFO - Reading all files from nextgen-passing
2023-07-13 13:32:30,911 - INFO -   + Reading nextgen-passing_2017.csv.gz
2023-07-13 13:32:30,917 - INFO -   + Reading nextgen-passing_2021.csv.gz
2023-07-13 13:32:30,928 - INFO -   + Reading nextgen-passing_2019.csv.gz
2023-07-13 13:32:30,932 - INFO -   + Reading nextgen-passing_2020.csv.gz
2023-07-13 13:32:30,936 - INFO -   + Reading nextgen-passing_2016.csv.gz
2023-07-13 13:32:30,939 - INFO -   + Reading nextgen-passing_2022.csv.gz
2023-07-13 13:32:30,943 - INFO -   + Reading nextgen-passing_2018.csv.gz


CPU times: user 26.8 ms, sys: 4.96 ms, total: 31.8 ms
Wall time: 38.2 ms


In [14]:
%%time
next_rec_df = load_files('nextgen-receiving')


2023-07-13 13:32:30,954 - INFO - Reading all files from nextgen-receiving
2023-07-13 13:32:30,955 - INFO -   + Reading nextgen-receiving_2021.csv.gz
2023-07-13 13:32:30,961 - INFO -   + Reading nextgen-receiving_2017.csv.gz
2023-07-13 13:32:30,967 - INFO -   + Reading nextgen-receiving_2019.csv.gz
2023-07-13 13:32:30,973 - INFO -   + Reading nextgen-receiving_2016.csv.gz
2023-07-13 13:32:30,980 - INFO -   + Reading nextgen-receiving_2020.csv.gz
2023-07-13 13:32:30,986 - INFO -   + Reading nextgen-receiving_2018.csv.gz
2023-07-13 13:32:30,992 - INFO -   + Reading nextgen-receiving_2022.csv.gz


CPU times: user 38.1 ms, sys: 7.61 ms, total: 45.7 ms
Wall time: 47.1 ms


In [15]:
%%time
next_rush_df = load_files('nextgen-rushing')

2023-07-13 13:32:31,004 - INFO - Reading all files from nextgen-rushing
2023-07-13 13:32:31,005 - INFO -   + Reading nextgen-rushing_2018.csv.gz
2023-07-13 13:32:31,009 - INFO -   + Reading nextgen-rushing_2022.csv.gz
2023-07-13 13:32:31,011 - INFO -   + Reading nextgen-rushing_2016.csv.gz
2023-07-13 13:32:31,014 - INFO -   + Reading nextgen-rushing_2020.csv.gz
2023-07-13 13:32:31,017 - INFO -   + Reading nextgen-rushing_2019.csv.gz
2023-07-13 13:32:31,020 - INFO -   + Reading nextgen-rushing_2021.csv.gz
2023-07-13 13:32:31,024 - INFO -   + Reading nextgen-rushing_2017.csv.gz


CPU times: user 20.7 ms, sys: 3.6 ms, total: 24.3 ms
Wall time: 24.2 ms


### <font color="#9370DB">players</font>

In [16]:
%%time
players_df = load_files('players')
players_df = transform_players(players_df)

2023-07-13 13:32:31,031 - INFO - Reading all files from players
2023-07-13 13:32:31,032 - INFO -   + Reading players.parquet
2023-07-13 13:32:31,065 - INFO - Process players dataset...
2023-07-13 13:32:31,065 - INFO - drop players without gsis_ids - they won't link to player_stats
2023-07-13 13:32:31,082 - INFO - fill empty players status to 'NONE'
2023-07-13 13:32:31,089 - INFO - rename gsis_id to player_id...


CPU times: user 62.4 ms, sys: 8.49 ms, total: 70.9 ms
Wall time: 61 ms


---

# <font color=teal>store to database so we can perform some SQL operations</font>

In [17]:
def load_all_datasets_to_db(data: dict):
    data['schema'] = database_schema
    load_dims_to_db(data)


In [18]:
%%time
if LOAD_TO_DB:
    datasets.update({
        'players': players_df,
        'player_stats': stats_df,
        'adv_stats_def': advstats_def_df,
        'adv_stats_pass': advstats_pass_df,
        'adv_stats_rec': advstats_rec_df,
        'adv_stats_rush': advstats_rush_df,
        'nextgen_pass': next_pass_df,
        'nextgen_rec': next_rec_df,
        'nextgen_rush': next_rush_df
    })
    load_all_datasets_to_db(datasets)

2023-07-13 13:32:31,098 - INFO - create table play_actions in schema controls
2023-07-13 13:33:08,596 - INFO - create table game_drive in schema controls
2023-07-13 13:33:30,072 - INFO - create table play_analytics in schema controls
2023-07-13 13:34:52,270 - INFO - create table play_situations in schema controls
2023-07-13 13:35:22,007 - INFO - create table play_metrics in schema controls
2023-07-13 13:35:45,309 - INFO - create table player_events in schema controls
2023-07-13 13:35:55,187 - INFO - create table game_info in schema controls
2023-07-13 13:35:55,358 - INFO - create table player_participation in schema controls
2023-07-13 13:41:10,379 - INFO - create table players in schema controls
2023-07-13 13:41:12,078 - INFO - create table player_stats in schema controls
2023-07-13 13:41:28,973 - INFO - create table adv_stats_def in schema controls
2023-07-13 13:41:29,364 - INFO - create table adv_stats_pass in schema controls
2023-07-13 13:41:29,419 - INFO - create table adv_stats_r

CPU times: user 3min 46s, sys: 1min 3s, total: 4min 50s
Wall time: 9min
